In [1]:
import pandas as pd
input_bouncer_data = pd.read_csv('./dataset/input_bouncer.csv')
output_bouncer_data = pd.read_csv("./dataset/random_sample_bouncer.csv")

In [2]:
input_bouncer_data["underspecified"].value_counts()

underspecified
1.0    653
2.0    542
0.0    396
3.0    108
Name: count, dtype: int64

In [3]:
input_bouncer_data["input_bounce"] = input_bouncer_data["underspecified"] >= 2

In [4]:
def parse_frac(ts):
    d = eval(ts)
    succ = len(d["FAIL_TO_PASS"]["success"]) + len(d["PASS_TO_PASS"]["success"])
    tot = succ + len(d["FAIL_TO_PASS"]["failure"]) + len(d["PASS_TO_PASS"]["failure"])
    return succ / tot if tot > 0 else 0.0


pass_frac = output_bouncer_data["tests_status"].apply(parse_frac)

In [5]:
output_bouncer_data["output_quality"] = pass_frac
output_bouncer_data["output_bounce"] = pass_frac < 1.0

In [6]:
output_bouncer_data["output_bounce"].value_counts()

output_bounce
True     405
False    237
Name: count, dtype: int64

In [8]:
output_bouncer_data = output_bouncer_data.set_index("instance_id")
input_bouncer_data = input_bouncer_data.set_index("instance_id")

In [9]:
input_bouncer_data = input_bouncer_data.rename(
    columns={
        "patch": "golden_patch",
        "underspecified": "input_quality",
    }
)
dropped_columns = output_bouncer_data.columns.intersection(input_bouncer_data.columns)
output_bouncer_data = output_bouncer_data.drop(columns=dropped_columns)
print("Dropped columns:", dropped_columns.tolist())

Dropped columns: ['problem_statement', 'repo', 'base_commit']


In [10]:
bouncer_bench_lite = input_bouncer_data.join(output_bouncer_data, how="inner")

In [12]:
bouncer_bench_lite["input_bounce"].value_counts()

input_bounce
False    539
True     103
Name: count, dtype: int64

In [13]:
bouncer_bench_lite["output_bounce"].value_counts()

output_bounce
True     405
False    237
Name: count, dtype: int64

In [14]:
bouncer_bench_lite.to_csv("./dataset/bouncer_bench_lite.csv", index=True)